# Generate all the relevant data for Ile de France region

In [1]:
import numpy as np
import pandas as pd

# Regions

In [2]:
regions = ["Ile-de-France"]

# Age groups

In [3]:
age_groups = ["age_group_0_9", "age_group_10_19","age_group_20_29","age_group_30_39",\
              "age_group_40_49","age_group_50_59","age_group_60_69","age_group_70_79","age_group_80_plus"]

print(age_groups)

['age_group_0_9', 'age_group_10_19', 'age_group_20_29', 'age_group_30_39', 'age_group_40_49', 'age_group_50_59', 'age_group_60_69', 'age_group_70_79', 'age_group_80_plus']


# Lockdown patterns

In [4]:
# Entries in the lockdown matrix ordered as follows: home, work, school, transport, leisure, otherplace
activities = ["home", "work", "school", "transport", "leisure", "other"]

# Dictionary based on age group and activity
lockdown_patterns = {}
for a in age_groups:
    lockdown_patterns[a] = pd.read_excel("./Final_Data/ile-de-france_data.xlsx",sheet_name=a)
    
print(lockdown_patterns)

# TEST: Print lockdown pattern coefficient for age_group_1 for school in scenario 0
lockdown_patterns["age_group_20_29"]["school"][0]
lockdown_patterns["age_group_20_29"]


{'age_group_0_9':    home  work  school  transport  leisure  other
0     1     0     0.0        0.0      0.0    0.0
1     1     1     0.5        0.5      0.0    0.0
2     1     1     1.0        1.0      0.0    0.0
3     1     1     1.0        1.0      0.5    0.5
4     1     1     1.0        1.0      1.0    1.0, 'age_group_10_19':    home  work  school  transport  leisure  other
0     1     0     0.0        0.0      0.0    0.0
1     1     1     0.5        0.5      0.0    0.0
2     1     1     1.0        1.0      0.0    0.0
3     1     1     1.0        1.0      0.5    0.5
4     1     1     1.0        1.0      1.0    1.0, 'age_group_20_29':    home  work  school  transport  leisure  other
0     1   0.0       0        0.0      0.0    0.0
1     1   0.5       1        0.5      0.0    0.0
2     1   1.0       1        1.0      0.0    0.0
3     1   1.0       1        1.0      0.5    0.5
4     1   1.0       1        1.0      1.0    1.0, 'age_group_30_39':    home  work  school  transport  leisur

,home,work,school,transport,leisure,other
0,1,0.0,0,0.0,0.0,0.0
1,1,0.5,1,0.5,0.0,0.0
2,1,1.0,1,1.0,0.0,0.0
3,1,1.0,1,1.0,0.5,0.5
4,1,1.0,1,1.0,1.0,1.0


# Economic & demographic data

#### Side note on approximating the economic values

The above is an approximation, based on the gross salary FTE data in file T404.xls. Based on the same data, we can do a better approximation for the economic values of groups 3 and 4. 

As an example, for group 4 we need an economic value for ages 25, 26, ..., 49, whereas file T404.xls provides a value for ages 26, 27, ..., 49. We assume the value 44617 in the file pertains to the age 37.5. We also assume value 24752 in the file pertains to age 22. Then we can do linear interpolation. To consider further...


#### Note on the full time equivalent employment rate 

(Source: https://www.insee.fr/fr/metadonnees/definition/c1743)

Le taux d'emploi en équivalent temps plein d'une classe d'individus est calculé en rapportant le nombre d'individus de la classe ayant un emploi converti en équivalent temps plein au nombre total d'individus dans la classe. Il peut être calculé sur l'ensemble de la population d'un pays, mais on se limite le plus souvent à la population en âge de travailler (généralement définie, en comparaison internationale, comme les personnes âgées de 15 à 64 ans), ou à une sous-catégorie de la population en âge de travailler (femmes de 25 à 29 ans par exemple).

Browser translation: "The full time equivalent employment rate is the ratio between the number of jobs converted into full-time and the total population. The number of people in full-time equivalent is obtained with the part-time rate recalculated in the Labour Force Survey: a person working half-time count for 0.5, or a person working 80% count for 0.8. This indicator is interesting because it shows how heterogeneous of working hours, because for the ILO, an hour during the reference week is enough to be considered as in employment."

In [5]:
# value generated when working
df_valwork = pd.read_excel("./Final_Data/ile-de-france_data.xlsx",sheet_name="value_working",index_col=0)
#print(df_valwork)

# fraction of population working
df_fracwork = pd.read_excel("./Final_Data/ile-de-france_data.xlsx",sheet_name="fraction_working",index_col=0)
#print(df_fracwork)

# population
df_population = pd.read_excel("./Final_Data/ile-de-france_data.xlsx",sheet_name="population",index_col=0)
#print(df_population)

# Parameter for number of days in a year. Do we want to account only for working days here?
days_in_year = 365

# Economic value from an individual from each age group being in non-lockdown, per region, over one day
value_age_group_region_day = {}
for r in regions:
    for a in age_groups:
        value_age_group_region_day[(r,a)] = df_fracwork[a][r] * df_valwork[a][r] / days_in_year

### Calculate an economic value multiplier for each group 
We do this for each possible tuple of (region,age,lockdown-pattern)

In [6]:
fraction_full_lockdown = 0.5   # the fraction of economic value the group generates in full lockdown

economic_values = {}
for r in regions:
    for a in age_groups:
        for i in range(0,len(lockdown_patterns[a].index)):
            economic_values[(r,a,i)] = \
            (lockdown_patterns[a]["work"][i] * 1.0 + (1-lockdown_patterns[a]["work"][i]) * fraction_full_lockdown ) * \
            value_age_group_region_day[(r,a)]

In [7]:
value_age_group_region_day

{('Ile-de-France', 'age_group_0_9'): 0.0,
 ('Ile-de-France', 'age_group_10_19'): 0.0,
 ('Ile-de-France', 'age_group_20_29'): 38.961495895150684,
 ('Ile-de-France', 'age_group_30_39'): 85.28878053728769,
 ('Ile-de-France', 'age_group_40_49'): 109.06185238356166,
 ('Ile-de-France', 'age_group_50_59'): 90.93383013698627,
 ('Ile-de-France', 'age_group_60_69'): 22.733457534246575,
 ('Ile-de-France', 'age_group_70_79'): 0.0,
 ('Ile-de-France', 'age_group_80_plus'): 0.0}

In [8]:
economic_values[("Ile-de-France","age_group_30_39",4)]

85.28878053728769

# Hospital and ICU capacity data
NOTE: Hospital number is completely fudged for now.

In [9]:
df_hospital_ICU = pd.read_excel("./Final_Data/ile-de-france_data.xlsx",sheet_name="ICU_hospital",index_col=0)
print(df_hospital_ICU)

for r in regions:
    print("ICU Cap: %.2f" % df_hospital_ICU["ICU"][r]) 
    print("Hospital Cap: %.2f" % df_hospital_ICU["Hospital"][r])

                ICU  Hospital
Ile-de-France  2500  20000000
ICU Cap: 2500.00
Hospital Cap: 20000000.00


# Social contact matrices

In [10]:
# read the social contract matrices for each activity as dictionaries
social_contact_matrices_df_weekdays = {}
social_contact_matrices_np_weekdays = {}

for a in activities:
    social_contact_matrices_df_weekdays[a] = pd.read_excel("./Final_Data/social_contact_matrices_weekdays.xlsx",sheet_name=a,index_col=0)
    social_contact_matrices_np_weekdays[a] = social_contact_matrices_df_weekdays[a].to_numpy()

In [11]:
social_contact_matrices_df_weekdays["work"]

,age_group_0_9,age_group_10_19,age_group_20_29,age_group_30_39,age_group_40_49,age_group_50_59,age_group_60_69,age_group_70_79,age_group_80_plus
age_group_0_9,0.490477,0.882075,0.715680,0.697605,0.540703,0.612702,0.382643,0.182921,0.053024
age_group_10_19,0.908284,1.495565,1.310695,1.276592,1.173656,1.037656,0.883404,0.474788,0.051500
age_group_20_29,0.706566,1.256665,1.683328,1.673736,1.511905,1.455544,0.798214,0.443829,0.147763
age_group_30_39,0.677085,1.203290,1.645460,2.036827,1.691207,1.267670,0.680321,0.199877,0.139763
age_group_40_49,0.486867,1.026306,1.378933,1.568972,1.705889,1.561871,0.584494,0.305998,0.075504
age_group_50_59,0.582581,0.958176,1.401843,1.241880,1.649303,1.553066,0.720788,0.333600,0.199014
age_group_60_69,0.456000,1.022385,0.963511,0.835317,0.773569,0.903381,0.468875,0.196595,0.059351
age_group_70_79,0.303415,0.764815,0.745685,0.341588,0.563689,0.581957,0.273636,0.180595,0.056367
age_group_80_plus,0.138734,0.130857,0.391597,0.376762,0.219395,0.547628,0.130307,0.088913,0.016936


In [12]:
social_contact_total_weekdays = np.zeros((len(age_groups),len(age_groups)))

for a in activities:
    social_contact_total_weekdays += social_contact_matrices_np_weekdays[a]

In [13]:
print(social_contact_total_weekdays)

[[4.00738005 1.78160922 1.32190379 2.04365945 1.59720855 1.12392243
  0.75160012 0.30579141 0.10150104]
 [1.83454467 9.60369497 2.30863404 2.18984362 2.77074639 1.5737782
  1.15973554 0.66536658 0.14091241]
 [1.3050687  2.21346561 4.81498301 2.91526474 2.65580905 2.43783106
  1.14857485 0.71017952 0.30149776]
 [1.98354657 2.06410191 2.86601428 3.67869557 2.89009648 2.1417177
  1.45535955 0.69045727 0.30293318]
 [1.43818156 2.42288623 2.42223016 2.68120826 3.45435849 2.67568085
  1.15143844 0.84448076 0.21174613]
 [1.06867075 1.45323305 2.34788953 2.0981471  2.82546417 3.03332866
  1.56782149 0.72921375 0.47383701]
 [0.89569035 1.34219061 1.38642647 1.78692981 1.52391135 1.96498886
  2.18671764 0.88250418 0.44176839]
 [0.50722189 1.0718101  1.19318373 1.1799827  1.55564457 1.27209623
  1.22833948 1.30008814 0.59280074]
 [0.26557033 0.35804901 0.79902367 0.81662385 0.61527957 1.30385855
  0.96991272 0.93507214 0.814026  ]]


### Function that calculates social contact matrix for every pair of triples

In [14]:
def calculate_contact(activities, age_groups, social_contact_matrices_df, lockdown_patterns,  \
                       ageg_row, idx_lockdown_row, ageg_col, idx_lockdown_col, mixing):
    # INPUTS:
    # Two lockdown profiles. A lockdown profile is a vector that defines the degree of each activity compared to 
    # normal level.
    # Mixing: 1: take the minimum of the coefficients for each activity. 0: multiply them.
    
    # OUTPUT:
    # A value for the contact matrix coefficient for two lockdown profiles.
    
    contact = 0
    for a in activities:
       
        if (mixing):
            contact += min( lockdown_patterns[ageg_row][a][idx_lockdown_row],\
                            lockdown_patterns[ageg_col][a][idx_lockdown_col] )*social_contact_matrices_df[a][ageg_col][ageg_row]
        else:
            contact += lockdown_patterns[ageg_row][a][idx_lockdown_row] * \
            lockdown_patterns[ageg_col][a][idx_lockdown_col] *social_contact_matrices_df[a][ageg_col][ageg_row]
    
    return(contact)

In [15]:
print(calculate_contact(activities, age_groups, social_contact_matrices_df_weekdays, lockdown_patterns,  \
                  "age_group_10_19", 0, "age_group_20_29", 0, 1))

print(calculate_contact(activities, age_groups, social_contact_matrices_df_weekdays, lockdown_patterns,  \
                  "age_group_10_19", 0, "age_group_20_29", 1, 1))

print(calculate_contact(activities, age_groups, social_contact_matrices_df_weekdays, lockdown_patterns,  \
                  "age_group_10_19", 1, "age_group_20_29", 1, 1))

0.304540942921224
0.304540942921224
1.114937477815619


In [16]:
def calculate_all_contacts(regions, activities, age_groups, social_contact_matrices_df, lockdown_patterns, mixing):
        
    all_contacts = {}
    for r in regions:
        for ar in age_groups:
            for ac in age_groups:
                for idx_lock_ar in range(0,len(lockdown_patterns[ar].index)):
                    for idx_lock_ac in range(0,len(lockdown_patterns[ac].index)):
                        all_contacts[(r,ar,idx_lock_ar,r,ac,idx_lock_ac)] = \
                        calculate_contact(activities, age_groups, social_contact_matrices_df, lockdown_patterns, \
                                          ar,idx_lock_ar,ac,idx_lock_ac,mixing)
    
    return(all_contacts)

In [17]:
all_contacts_weekdays = calculate_all_contacts(regions, activities, age_groups, social_contact_matrices_df_weekdays, lockdown_patterns, 1)

In [18]:
# read the social contract matrices for each activity as dictionaries
social_contact_matrices_df_weekends = {}
social_contact_matrices_np_weekends = {}

for a in activities:
    social_contact_matrices_df_weekends[a] = pd.read_excel("./Final_Data/social_contact_matrices_weekends.xlsx",sheet_name=a,index_col=0)
    social_contact_matrices_np_weekends[a] = social_contact_matrices_df_weekends[a].to_numpy()

In [19]:
social_contact_total_weekends = np.zeros((len(age_groups),len(age_groups)))

for a in activities:
    social_contact_total_weekends += social_contact_matrices_np_weekends[a]

In [20]:
all_contacts_weekends = calculate_all_contacts(regions, activities, age_groups, social_contact_matrices_df_weekends, lockdown_patterns, 1)


In [21]:
# read the social contract matrices for each activity as dictionaries
social_contact_matrices_df_new = {}
social_contact_matrices_np_new = {}

for a in activities:
    social_contact_matrices_df_new[a] = pd.read_excel("./Final_Data/social_contact_matrices_new.xlsx",sheet_name=a,index_col=0)
    social_contact_matrices_np_new[a] = social_contact_matrices_df_new[a].to_numpy()
    
social_contact_total_new = np.zeros((len(age_groups),len(age_groups)))

for a in activities:
    social_contact_total_new += social_contact_matrices_np_new[a]
    
all_contacts_new = calculate_all_contacts(regions, activities, age_groups, social_contact_matrices_df_new, lockdown_patterns, 1)


In [22]:
# read the social contract matrices for each activity as dictionaries
social_contact_matrices_df_old = {}
social_contact_matrices_np_old = {}

for a in activities:
    social_contact_matrices_df_old[a] = pd.read_excel("./Final_Data/social_contact_matrices.xlsx",sheet_name=a,index_col=0)
    social_contact_matrices_np_old[a] = social_contact_matrices_df_old[a].to_numpy()
    
social_contact_total_old = np.zeros((len(age_groups),len(age_groups)))

for a in activities:
    social_contact_total_old += social_contact_matrices_np_old[a]
    
all_contacts_old = calculate_all_contacts(regions, activities, age_groups, social_contact_matrices_df_old, lockdown_patterns, 1)


In [23]:
# read the social contract matrices for each activity as dictionaries
social_contact_matrices_df_spc = {}
social_contact_matrices_np_spc = {}

for a in activities:
    social_contact_matrices_df_spc[a] = pd.read_excel("./Final_Data/social_contact_matrices_SPC.xlsx",sheet_name=a,index_col=0)
    social_contact_matrices_np_spc[a] = social_contact_matrices_df_spc[a].to_numpy()
    
social_contact_total_spc = np.zeros((len(age_groups),len(age_groups)))

for a in activities:
    social_contact_total_spc += social_contact_matrices_np_spc[a]
    
all_contacts_spc = calculate_all_contacts(regions, activities, age_groups, social_contact_matrices_df_spc, lockdown_patterns, 1)


# SIR Parameters

In [24]:
R0 = 2.9
df_SEIR = pd.read_excel("./Final_Data/ile-de-france_data.xlsx",sheet_name="SEIR_params",index_col=0)

print(df_SEIR)

                   R_0    mu  sigma   p_ss  p_ICU_cond_ss     p_ICU       p_H  \
age_group_0_9      2.9  0.25   0.25  0.002          0.222  0.000444  0.001556   
age_group_10_19    2.9  0.25   0.25  0.002          0.222  0.000444  0.001556   
age_group_20_29    2.9  0.25   0.25  0.006          0.115  0.000690  0.005310   
age_group_30_39    2.9  0.25   0.25  0.013          0.159  0.002067  0.010933   
age_group_40_49    2.9  0.25   0.25  0.017          0.222  0.003774  0.013226   
age_group_50_59    2.9  0.25   0.25  0.035          0.276  0.009660  0.025340   
age_group_60_69    2.9  0.25   0.25  0.071          0.308  0.021868  0.049132   
age_group_70_79    2.9  0.25   0.25  0.113          0.249  0.028137  0.084863   
age_group_80_plus  2.9  0.25   0.25  0.320          0.056  0.017920  0.302080   

                   p_death_cond_ss  p_recov_cond_ss  lambda_H  lambda_HR  \
age_group_0_9                0.006            0.994  0.076278   0.075820   
age_group_10_19              0.006   

### Generate beta

In [25]:
def get_beta(df_SEIR, R0, contact_matrix) :
    
    num_groups = len(contact_matrix)
    sigma = df_SEIR["sigma"].to_numpy()
    mu = df_SEIR["mu"].to_numpy()  
    
    # compute F/beta and V
    F_over_beta = np.block([ [np.zeros((num_groups,num_groups)),contact_matrix],\
                            [np.zeros((num_groups,num_groups)), np.zeros((num_groups,num_groups))]])       
    V = np.block([[-np.diagflat(sigma),np.zeros((num_groups,num_groups))],\
                  [np.diagflat(sigma), -np.diagflat(mu)]])

    # determine eigenvalues of  -F/beta * V^(-1)
    eigval, eigvec = np.linalg.eig(np.matmul(-F_over_beta,np.linalg.inv(V)))

    # calculate beta
    beta = R0 / np.max(np.abs(eigval))
    return beta

In [26]:
beta_weekdays = get_beta(df_SEIR, 2.9, social_contact_total_weekdays)
print(beta_weekdays)

0.04240988982281467


In [27]:
beta_weekends = get_beta(df_SEIR, 2.9, social_contact_total_weekends)
print(beta_weekends)

0.06115986360123594


In [28]:
beta_new = get_beta(df_SEIR, 2.9, social_contact_total_new)
print(beta_new)

0.068932660782469


In [29]:
beta_old = get_beta(df_SEIR, 2.9, social_contact_total_old)
print(beta_old)

0.060606045273519614


In [30]:
beta_spc = get_beta(df_SEIR, 2.9, social_contact_total_spc)
print(beta_spc)

beta = beta_spc

0.03923794546657925


#### Calculate Relative Risk of Infection / Attack Rate per age-group
Done for purposes of comparing with Salje et al. Reference age group is 30-39. 

# Death Cost

In [31]:
df_death = pd.read_excel("./Final_Data/ile-de-france_data.xlsx",sheet_name="death_cost",index_col=0)

# Testing Capabilities

In [32]:
testing = {
    "Ile-de-France":
        {
            'antibodies':3000,
            'molecular':3000,
        }
}

# Export Data

In [35]:
data_dict = {
    "regions":regions,
    "age_groups":age_groups,
    "capacity":df_hospital_ICU,
    "all_contacts_weekdays":all_contacts_weekdays,
    "all_contacts_weekends":all_contacts_weekends,
    "all_contacts_new":all_contacts_new,
    "all_contacts_old":all_contacts_old,
    "all_contacts_spc":all_contacts_spc,
    "SEIR_parameters":df_SEIR,
    "beta":beta,
    "population":df_population,
    'death_cost':df_death['death_cost'],
    "testing":testing,
    "hospital_icu":df_hospital_ICU,
    "social_contact_matrices_weekdays":social_contact_matrices_df_weekdays,
    "social_contact_matrices_weekends":social_contact_matrices_df_weekends,
    "social_contact_matrices_new":social_contact_matrices_df_new,
    "social_contact_matrices_old":social_contact_matrices_df_old,
    "social_contact_matrices_spc":social_contact_matrices_df_spc,
    "activities":activities,
    "economic_value":value_age_group_region_day,
    "lockdown_fraction":fraction_full_lockdown,
    "lockdown_patterns":lockdown_patterns,
}

In [36]:
import pickle
pickle.dump( data_dict, open( "data_dict.p", "wb" ) )

In [37]:
df_population.sum()

age_group_0_9        1594856
age_group_10_19      1546109
age_group_20_29      1651917
age_group_30_39      1757848
age_group_40_49      1671811
age_group_50_59      1538636
age_group_60_69      1177841
age_group_70_79       792962
age_group_80_plus     546230
TOTAL                9761177
dtype: int64

In [38]:
df_SEIR

,R_0,mu,sigma,p_ss,p_ICU_cond_ss,p_ICU,p_H,p_death_cond_ss,p_recov_cond_ss,lambda_H,lambda_HR,lambda_HD,lambda_ICU,lambda_ICUR,lambda_ICUD
age_group_0_9,2.9,0.25,0.25,0.002,0.222,0.000444,0.001556,0.006,0.994,0.076278,0.075820,0.000458,0.052438,0.052124,0.000315
age_group_10_19,2.9,0.25,0.25,0.002,0.222,0.000444,0.001556,0.006,0.994,0.076278,0.075820,0.000458,0.052438,0.052124,0.000315
age_group_20_29,2.9,0.25,0.25,0.006,0.115,0.000690,0.005310,0.011,0.989,0.076278,0.075439,0.000839,0.052438,0.051862,0.000577
age_group_30_39,2.9,0.25,0.25,0.013,0.159,0.002067,0.010933,0.019,0.981,0.076278,0.074828,0.001449,0.052438,0.051442,0.000996
age_group_40_49,2.9,0.25,0.25,0.017,0.222,0.003774,0.013226,0.033,0.967,0.076278,0.073760,0.002517,0.052438,0.050708,0.001730
age_group_50_59,2.9,0.25,0.25,0.035,0.276,0.009660,0.025340,0.065,0.935,0.076278,0.071320,0.004958,0.052438,0.049030,0.003408
age_group_60_69,2.9,0.25,0.25,0.071,0.308,0.021868,0.049132,0.126,0.874,0.076278,0.066667,0.009611,0.052438,0.045831,0.006607
age_group_70_79,2.9,0.25,0.25,0.113,0.249,0.028137,0.084863,0.210,0.790,0.076278,0.060259,0.016018,0.052438,0.041426,0.011012
age_group_80_plus,2.9,0.25,0.25,0.320,0.056,0.017920,0.302080,0.316,0.684,0.076278,0.052174,0.024104,0.052438,0.035868,0.016571
